# Understanding Retrivals

- Suff Method
- MapReduce Method 

In [2]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS


In [3]:
#load openAI api key
os.environ['OPENAI_API_KEY'] = 'sk-proj-VZBi9kjb1DmxxDXHa0h2T3BlbkFJOPfgDcZ8ttJ9i1O3mH6S'

In [4]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Initialise LLM with required params
llm = OpenAI(temperature=0.9, max_tokens=500) 

/var/folders/wm/hlcxs9b97t9gs5wqfz71vw880000gn/T/ipykernel_4335/263616180.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9, max_tokens=500)


In [6]:
loaders = UnstructuredURLLoader(
    urls = [
        "https://www.moneycontrol.com/news/business/stocks/accumulate-tata-motors-target-of-rs-1075-prabhudas-lilladher-12538061.html",
        "https://www.moneycontrol.com/news/business/stocks/buy-tata-motors-target-of-rs-1188-sharekhan-12411611.html",
        "https://www.moneycontrol.com/news/business/stocks/reduce-tata-motors-target-of-rs-901-icici-securities-12411521.html"
        ]
)
data = loaders.load() 
len(data)

AttributeError: module 'cv2.dnn' has no attribute 'DictValue'

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [ ]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [ ]:
pkl = vectorindex_openai.serialize_to_bytes()

In [ ]:
# Storing vector index create in local
file_path="vector_index_1.pkl"
with open(file_path, "wb") as f:
    pickle.dump(pkl, f)

In [ ]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [ ]:
vectorIndex = FAISS.deserialize_from_bytes(embeddings=OpenAIEmbeddings(), serialized=vectorIndex,allow_dangerous_deserialization=True)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

In [ ]:
query = "What is the least and highest suggested value for tata motors"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)